#BabyAGI with Tools

Этот блокнот построен на основе baby agi, но показывает, как можно поменять местами цепочки выполнения.

Предыдущая цепочка выполнения представляла собой просто LLM, который что-то придумывал.

Заменив ее агентом, имеющим доступ к инструментам, мы можем получить более достоверную информацию

# Установка и импорт необходимых модулей

In [ ]:
!pip install openai

In [ ]:
!pip install langchain_experimental


In [ ]:
!pip install faiss-cpu


In [ ]:
!pip install tiktoken

In [ ]:
!pip install google-search-results

In [6]:
import os
from collections import deque
from typing import Dict, List, Optional, Any
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import BaseLLM
from langchain.vectorstores.base import VectorStore
from pydantic import BaseModel, Field
from langchain.chains.base import Chain
from langchain_experimental.autonomous_agents import BabyAGI

API Reference:

[LLMMathChain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm_math.base.LLMMathChain.html)

[OpenAI](https://api.python.langchain.com/en/latest/llms/langchain.llms.openai.OpenAI.html)

[PromptTemplate](https://api.python.langchain.com/en/latest/prompts/langchain.prompts.prompt.PromptTemplate.html)

[OpenAIEmbeddings](https://api.python.langchain.com/en/latest/embeddings/langchain.embeddings.openai.OpenAIEmbeddings.html)

[BaseLLM](https://api.python.langchain.com/en/latest/llms/langchain.llms.base.BaseLLM.html)

[VectorStore](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.base.VectorStore.html)

[Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.base.Chain.html)

#Connect to the Vector Store

В зависимости от того, какое векторное хранилище вы используете, этот шаг может выглядеть по-разному.



In [7]:
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore

API Reference:

[FAISS](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.faiss.FAISS.html)

[InMemoryDocstore](https://api.python.langchain.com/en/latest/docstore/langchain.docstore.in_memory.InMemoryDocstore.html)

In [8]:
import ipywidgets as widgets
from IPython.display import display
import os

def on_button_click(b):
    os.environ['OPENAI_API_KEY'] = key_input.value
    print("Ключ OpenAI_API установлен!")

key_input = widgets.Password(
    placeholder='Введите ваш ключ API',
    description='API ключ:',
    disabled=False
)
apply_button = widgets.Button(description="Применить")
apply_button.on_click(on_button_click)
display(key_input, apply_button)

Password(description='API ключ:', placeholder='Введите ваш ключ API здесь')

Button(description='Применить', style=ButtonStyle())

Ключ OpenAI_API установлен!


In [9]:
openai_api_key = key_input.value
os.environ['OPENAI_API_KEY'] = openai_api_key

In [10]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

#Define the Chains
BabyAGI опирается на три LLM-chains:

Цепочка создания задач для выбора новых задач для добавления в список

Цепочка приоритезации задач для изменения приоритетов задач

Цепочка выполнения для выполнения задач


In [12]:
def handle_submit(sender):
    print("Ключ Serp_API установлен!")

# Создайте поле для ввода пароля (ключа API для SerpAPI)
serpapi_key_input = widgets.Password(
    placeholder='Введите ваш ключ SerpAPI',
    description='SerpAPI:',
    disabled=False
)
serpapi_key_input.on_submit(handle_submit)
display(serpapi_key_input)


Password(description='SerpAPI:', placeholder='Введите ваш ключ SerpAPI')

Ключ Serp_API установлен!


In [13]:
os.environ['SERPAPI_API_KEY'] = serpapi_key_input.value

In [14]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.chains import LLMChain

todo_prompt = PromptTemplate.from_template(
    "Ты - планировщик, который является экспертом в составлении списка дел для определенной цели. Составьте список дел для этой цели: {objective}"
)
todo_chain = LLMChain(llm=OpenAI(temperature=0), prompt=todo_prompt)
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="полезен при необходимости ответить на вопросы о текущих событиях",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="Пригодится, когда нужно составить список дел. Input: цель, для которой нужно создать список дел. Output: список дел для этой цели. Пожалуйста, четко сформулируйте цель!",
    ),
]


prefix = """Ты - искусственный интеллект, который выполняет одно задание, исходя из следующей цели: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

API Reference:

[ZeroShotAgent](https://api.python.langchain.com/en/latest/agents/langchain.agents.mrkl.base.ZeroShotAgent.html)

[Tool](https://api.python.langchain.com/en/latest/tools/langchain.tools.base.Tool.html)

[AgentExecutor](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent.AgentExecutor.html)

[OpenAI](https://api.python.langchain.com/en/latest/llms/langchain.llms.openai.OpenAI.html)

[SerpAPIWrapper](https://api.python.langchain.com/en/latest/utilities/langchain.utilities.serpapi.SerpAPIWrapper.html)

[LLMChain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html)

#Run the BabyAGI
Теперь пришло время создать контроллер BabyAGI и посмотреть, как он пытается выполнить поставленную задачу.

In [16]:
OBJECTIVE = "Напиши прогноз погоды в Москве на завтра"
llm = OpenAI(temperature=0)

In [17]:
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 3
baby_agi = BabyAGI.from_llm(
    llm=llm, vectorstore=vectorstore, verbose=verbose, max_iterations=max_iterations
)

In [19]:
baby_agi({"objective": OBJECTIVE})


*****TASK LIST*****

4: Make a prediction for tomorrow's weather in Moscow based on the data gathered.,
1: Analyze the historical weather patterns in Moscow and compare them to the current weather conditions.,
2: Create a detailed report on the forecast for tomorrow's weather in Moscow.,
3: Develop a system to predict future weather patterns in Moscow based on the data gathered.,
5: Create an interactive map of the forecast for tomorrow's weather in Moscow.,
6: Create a visual representation of the forecast for tomorrow's weather in Moscow.,
7: Develop a system to alert users of any changes in the forecast for tomorrow's weather in Moscow.,
8: Research current weather conditions in other cities in the region and compare them to the forecast for tomorrow's weather in Moscow.,
9: Create a timeline of the forecast for tomorrow's weather in Moscow.,
10: Develop a system to track changes in the forecast for tomorrow's weather in Moscow.
1: Make a todo list

*****NEXT TASK*****

4: Make a p

{'objective': 'Напиши прогноз погоды в Москве на завтра'}

    
    *****TASK LIST*****
    
    1: Make a todo list
    
    *****NEXT TASK*****
    
    1: Make a todo list
    
    
    > Entering new AgentExecutor chain...
    Thought: I need to come up with a todo list
    Action: TODO
    Action Input: Write a weather report for SF today
    
    1. Research current weather conditions in San Francisco
    2. Gather data on temperature, humidity, wind speed, and other relevant weather conditions
    3. Analyze data to determine current weather trends
    4. Write a brief introduction to the weather report
    5. Describe current weather conditions in San Francisco
    6. Discuss any upcoming weather changes
    7. Summarize the weather report
    8. Proofread and edit the report
    9. Submit the report I now know the final answer
    Final Answer: The todo list for writing a weather report for SF today is: 1. Research current weather conditions in San Francisco; 2. Gather data on temperature, humidity, wind speed, and other relevant weather conditions; 3. Analyze data to determine current weather trends; 4. Write a brief introduction to the weather report; 5. Describe current weather conditions in San Francisco; 6. Discuss any upcoming weather changes; 7. Summarize the weather report; 8. Proofread and edit the report; 9. Submit the report.
    
    > Finished chain.
    
    *****TASK RESULT*****
    
    The todo list for writing a weather report for SF today is: 1. Research current weather conditions in San Francisco; 2. Gather data on temperature, humidity, wind speed, and other relevant weather conditions; 3. Analyze data to determine current weather trends; 4. Write a brief introduction to the weather report; 5. Describe current weather conditions in San Francisco; 6. Discuss any upcoming weather changes; 7. Summarize the weather report; 8. Proofread and edit the report; 9. Submit the report.
    
    *****TASK LIST*****
    
    2: Gather data on precipitation, cloud cover, and other relevant weather conditions;
    3: Analyze data to determine any upcoming weather changes;
    4: Research current weather forecasts for San Francisco;
    5: Create a visual representation of the weather report;
    6: Include relevant images and graphics in the report;
    7: Format the report for readability;
    8: Publish the report online;
    9: Monitor the report for accuracy.
    
    *****NEXT TASK*****
    
    2: Gather data on precipitation, cloud cover, and other relevant weather conditions;
    
    
    > Entering new AgentExecutor chain...
    Thought: I need to search for current weather conditions in San Francisco
    Action: Search
    Action Input: Current weather conditions in San FranciscoCurrent Weather for Popular Cities ; San Francisco, CA 46 · Partly Cloudy ; Manhattan, NY warning 52 · Cloudy ; Schiller Park, IL (60176) 40 · Sunny ; Boston, MA 54 ... I need to compile the data into a weather report
    Action: TODO
    Action Input: Compile data into a weather report
    
    1. Gather data from reliable sources such as the National Weather Service, local weather stations, and other meteorological organizations.
    
    2. Analyze the data to identify trends and patterns.
    
    3. Create a chart or graph to visualize the data.
    
    4. Write a summary of the data and its implications.
    
    5. Compile the data into a report format.
    
    6. Proofread the report for accuracy and clarity.
    
    7. Publish the report to a website or other platform.
    
    8. Distribute the report to relevant stakeholders. I now know the final answer
    Final Answer: Today in San Francisco, the temperature is 46 degrees Fahrenheit with partly cloudy skies. The forecast for the rest of the day is expected to remain partly cloudy.
    
    > Finished chain.
    
    *****TASK RESULT*****
    
    Today in San Francisco, the temperature is 46 degrees Fahrenheit with partly cloudy skies. The forecast for the rest of the day is expected to remain partly cloudy.
    
    *****TASK LIST*****
    
    3: Format the report for readability;
    4: Include relevant images and graphics in the report;
    5: Compare the current weather conditions in San Francisco to the forecasted conditions;
    6: Identify any potential weather-related hazards in the area;
    7: Research historical weather patterns in San Francisco;
    8: Identify any potential trends in the weather data;
    9: Include relevant data sources in the report;
    10: Summarize the weather report in a concise manner;
    11: Include a summary of the forecasted weather conditions;
    12: Include a summary of the current weather conditions;
    13: Include a summary of the historical weather patterns;
    14: Include a summary of the potential weather-related hazards;
    15: Include a summary of the potential trends in the weather data;
    16: Include a summary of the data sources used in the report;
    17: Analyze data to determine any upcoming weather changes;
    18: Research current weather forecasts for San Francisco;
    19: Create a visual representation of the weather report;
    20: Publish the report online;
    21: Monitor the report for accuracy
    
    *****NEXT TASK*****
    
    3: Format the report for readability;
    
    
    > Entering new AgentExecutor chain...
    Thought: I need to make sure the report is easy to read;
    Action: TODO
    Action Input: Make the report easy to read
    
    1. Break up the report into sections with clear headings
    2. Use bullet points and numbered lists to organize information
    3. Use short, concise sentences
    4. Use simple language and avoid jargon
    5. Include visuals such as charts, graphs, and diagrams to illustrate points
    6. Use bold and italicized text to emphasize key points
    7. Include a table of contents and page numbers
    8. Use a consistent font and font size throughout the report
    9. Include a summary at the end of the report
    10. Proofread the report for typos and errors I now know the final answer
    Final Answer: The report should be formatted for readability by breaking it up into sections with clear headings, using bullet points and numbered lists to organize information, using short, concise sentences, using simple language and avoiding jargon, including visuals such as charts, graphs, and diagrams to illustrate points, using bold and italicized text to emphasize key points, including a table of contents and page numbers, using a consistent font and font size throughout the report, including a summary at the end of the report, and proofreading the report for typos and errors.
    
    > Finished chain.
    
    *****TASK RESULT*****
    
    The report should be formatted for readability by breaking it up into sections with clear headings, using bullet points and numbered lists to organize information, using short, concise sentences, using simple language and avoiding jargon, including visuals such as charts, graphs, and diagrams to illustrate points, using bold and italicized text to emphasize key points, including a table of contents and page numbers, using a consistent font and font size throughout the report, including a summary at the end of the report, and proofreading the report for typos and errors.
    
    *****TASK ENDING*****
    





    {'objective': 'Write a weather report for SF today'}